# Notes

Issues:
- START PARAMETER??? 
- Modell definieren wie Chauvet (1998)
- Daten 

# Init

In [2]:
import numpy as np
from numpy.testing import assert_allclose
from statsmodels.tsa.statespace.regime_switching.tools import \
        MarkovSwitchingParams
from statsmodels.tsa.statespace.regime_switching.switching_dynamic_factor \
        import _DynamicFactorWithFactorIntercept, SwitchingDynamicFactor

class KimYoo1995NonswitchingModel(_DynamicFactorWithFactorIntercept):
    '''
    This is dynamic factor model with some restrictions on parameters.
    See http://econ.korea.ac.kr/~cjkim/MARKOV/programs/sw_ms.opt.
    '''

    def __init__(self, endog, k_factors, factor_order, **kwargs):

        super(KimYoo1995NonswitchingModel, self).__init__(endog, k_factors,
                factor_order, **kwargs)

        k_endog = self.k_endog
        error_order = self.error_order

        offset = 0
        self._gamma_idx = np.s_[offset:offset + k_endog + k_factors - 1]
        offset += k_endog + k_factors - 1
        self._phi_idx = np.s_[offset:offset + error_order]
        offset += error_order
        self._psi_idx = np.s_[offset:offset + k_endog * error_order]
        offset += k_endog * error_order
        self._sigma_idx = np.s_[offset:offset + k_endog]
        offset += k_endog
        self._mu_idx = np.s_[offset]
        self._params_without_intercept_idx = np.s_[:offset]
        self._kimyoo_k_params_without_intercept = offset
        offset += 1

        self.kimyoo_k_params = offset

        # For the sake of clarity
        self._base_class_k_params = self.k_params_with_factor_intercept

    def _get_dynamic_factor_params(self, params_without_intercept):
        '''
        params_without_intercept - just a prefix of parameters vector, since
        intercept is the last value
        '''

        dtype = self.ssm.dtype
        k_endog = self.k_endog
        k_factors = self.k_factors
        factor_order = self.factor_order
        error_order = self.error_order

        dynamic_factor_params = np.zeros((self.dynamic_factor_k_params,),
                dtype=dtype)

        # 1. Factor loadings

        factor_loadings_matrix = np.zeros((k_endog, k_factors), dtype=dtype)

        gammas = params_without_intercept[self._gamma_idx]

        factor_loadings_matrix[:, 0] = gammas[:k_endog]
        factor_loadings_matrix[-1, 1:] = gammas[k_endog:]

        dynamic_factor_params[self._params_loadings] = \
                factor_loadings_matrix.ravel()

        # 2. Error covs

        dynamic_factor_params[self._params_error_cov] = \
                params_without_intercept[self._sigma_idx]

        # 3. Factor transition

        phi = params_without_intercept[self._phi_idx]


        # `factor_order` == 1, so this essentially is a square matrix
        factor_transition_params = np.zeros((k_factors,
                k_factors * factor_order), dtype=dtype)

        # TODO: check order of parameters

        factor_transition_params[0, :k_factors - 1] = phi
        factor_transition_params[1:, :-1] = np.identity(k_factors - 1,
                dtype=dtype)

        dynamic_factor_params[self._params_factor_transition] = \
                factor_transition_params.ravel()

        # 4. Error transition

        psi = params_without_intercept[self._psi_idx]

        dynamic_factor_params[self._params_error_transition] = psi

        return dynamic_factor_params

    def _get_params_without_intercept(self, dynamic_factor_params):
        '''
        reverse to previous
        '''

        dtype = self.ssm.dtype
        k_endog = self.k_endog
        k_factors = self.k_factors
        factor_order = self.factor_order
        error_order = self.error_order

        params_without_intercept = np.zeros(( \
                self._kimyoo_k_params_without_intercept,),
                dtype=dtype)

        # 1. Factor loadings

        factor_loadings_matrix = \
                dynamic_factor_params[self._params_loadings].reshape( \
                k_endog, k_factors)

        gammas = np.zeros((k_endog + k_factors - 1,), dtype=dtype)

        gammas[:k_endog] = factor_loadings_matrix[:, 0]
        gammas[k_endog:] = factor_loadings_matrix[-1, 1:]

        params_without_intercept[self._gamma_idx] = gammas

        # 2. Error covs

        params_without_intercept[self._sigma_idx] = dynamic_factor_params[ \
                self._params_error_cov]

        # 3. Factor transition

        # `factor_order` == 1, so this essentially is a square matrix
        factor_transition_params = \
                dynamic_factor_params[self._params_factor_transition \
                ].reshape(k_factors, k_factors * factor_order)

        params_without_intercept[self._phi_idx] = factor_transition_params[0,
                :k_factors - 1]

        # 4. Error transition

        psi = dynamic_factor_params[self._params_error_transition]

        params_without_intercept[self._psi_idx] = psi

        return params_without_intercept

    def _get_base_class_params(self, params):

        dtype = self.ssm.dtype

        base_class_params = np.zeros((self._base_class_k_params,), dtype=dtype)

        base_class_params[self._dynamic_factor_params_idx] = \
                self._get_dynamic_factor_params(
                params[self._params_without_intercept_idx])

        base_class_params[self._factor_intercept_idx] = params[self._mu_idx]

        return base_class_params

    def _get_params(self, base_class_params):

        dtype = self.ssm.dtype

        params_without_intercept = self._get_params_without_intercept(
                base_class_params[self._dynamic_factor_params_idx])

        params = np.zeros((self.kimyoo_k_params,), dtype=dtype)

        params[self._params_without_intercept_idx] = params_without_intercept

        params[self._mu_idx] = base_class_params[self._factor_intercept_idx]

        return params

    @property
    def start_params(self):

        base_start_params = super(KimYoo1995NonswitchingModel,
                self).start_params

        return self._get_params(base_start_params)

    def transform_params(self, unconstrained):

        unconstr_base_class_params = self._get_base_class_params(unconstrained)

        constr_base_class_params = super(KimYoo1995NonswitchingModel,
                self).transform_params(unconstr_base_class_params)

        constrained = self._get_params(constr_base_class_params)

        return constrained

    def untransform_params(self, constrained):

        constr_base_class_params = self._get_base_class_params(constrained)

        unconstr_base_class_params = super(KimYoo1995NonswitchingModel,
                self).untransform_params(constr_base_class_params)

        unconstrained = self._get_params(unconstr_base_class_params)

        return unconstrained

    def update(self, params, **kwargs):

        dtype = self.ssm.dtype
        k_states = self.k_states

        base_class_params = self._get_base_class_params(params)

        super(KimYoo1995NonswitchingModel, self).update(base_class_params,
                **kwargs)

        # Filter initialization.

        state_intercept = self['state_intercept']

        transition = self['transition']

        raveled_state_cov = (self['selection'].dot(self['state_cov']).dot(
                self['selection'].T)).reshape(-1, 1)

        initial_state = np.linalg.inv(np.identity(k_states, dtype=dtype) - \
                transition).dot(state_intercept).ravel()

        transition_outer_sqr = np.zeros((k_states * k_states,
                k_states * k_states), dtype=dtype)

        for i in range(k_states):
            for j in range(k_states):
                transition_outer_sqr[i * k_states:i * k_states + k_states,
                        j * k_states:j * k_states + k_states] = \
                        transition * transition[i, j]

        initial_state_cov = np.linalg.inv(np.identity(k_states * k_states,
                dtype=dtype) - transition_outer_sqr).dot(raveled_state_cov
                ).reshape(k_states, k_states).T

        self.initialize_known(initial_state, initial_state_cov)


class KimYoo1995Model(SwitchingDynamicFactor):
    '''
    This is switching dynamic factor model with some restrictions on
    parameters. See http://econ.korea.ac.kr/~cjkim/MARKOV/programs/sw_ms.opt.
    '''

    def __init__(self, k_regimes, endog, k_factors, factor_order, **kwargs):

        super(KimYoo1995Model, self).__init__(k_regimes, endog, k_factors,
                factor_order, **kwargs)

        # we need this instance because of its useful methods
        # `_get_dynamic_factor_params`, `_get_params_without_intercept` and
        # others
        self._nonswitching_model = KimYoo1995NonswitchingModel(endog,
                k_factors, factor_order, **kwargs)

        # For the sake of clarity
        self._base_class_parameters = self.parameters

        # params vector for this model differs from params vector in
        # `SwitchingDynamicFactor`.
        self._kimyoo_parameters = MarkovSwitchingParams(k_regimes)

        self._kimyoo_parameters['regime_transition'] = [False] * k_regimes * \
                (k_regimes - 1)

        # Number of nonswitching params is equal to number of parameters in
        # nonswitching model, except of factor intercept (1 value).
        self._kimyoo_parameters['nonswitching_params'] = [False] * \
                (self._nonswitching_model.kimyoo_k_params - 1)

        self._kimyoo_parameters['factor_intercept'] = [True]

        # A dirty hack, required, because Kim-Yoo model's specification is a
        # little different from Statsmodels one.
        self['state_cov', :k_factors, :k_factors] = 0
        self['state_cov', 0, 0] = 1

    def _get_base_class_params(self, params):

        dtype = self.ssm.dtype

        base_class_params = np.zeros((self._base_class_parameters.k_params,),
                dtype=dtype)

        base_class_params[self._base_class_parameters['regime_transition']] = \
                params[self._kimyoo_parameters['regime_transition']]

        params_without_intercept = params[self._kimyoo_parameters[
                'nonswitching_params']]

        base_class_params[self._base_class_parameters['dynamic_factor']] = \
                self._nonswitching_model._get_dynamic_factor_params(
                params_without_intercept)

        base_class_params[self._base_class_parameters['factor_intercept']] = \
                params[self._kimyoo_parameters['factor_intercept']]

        return base_class_params

    def _get_params(self, base_class_params):

        dtype = self.ssm.dtype

        params = np.zeros((self._kimyoo_parameters.k_params,), dtype=dtype)

        params[self._kimyoo_parameters['regime_transition']] = \
                base_class_params[self._base_class_parameters[
                'regime_transition']]

        dynamic_factor_params = base_class_params[self._base_class_parameters[
                'dynamic_factor']]

        params[self._kimyoo_parameters['nonswitching_params']] = \
                self._nonswitching_model._get_params_without_intercept(
                dynamic_factor_params)

        params[self._kimyoo_parameters['factor_intercept']] = \
                base_class_params[self._base_class_parameters[
                'factor_intercept']]

        return params

    @property
    def start_params(self):

        dtype = self.ssm.dtype

        base_start_params = super(KimYoo1995Model, self).start_params

        return self._get_params(base_start_params)

    def get_nonswitching_model(self):

        # don't need to instantiate a new model, since we already have one
        return self._nonswitching_model

    def update_params(self, params, nonswitching_model_params):

        base_class_params = self._get_base_class_params(params)

        nonswitching_base_class_params = \
                self._nonswitching_model._get_base_class_params(
                nonswitching_model_params)

        updated_base_class_params = super(KimYoo1995Model,
                self).update_params(base_class_params,
                nonswitching_base_class_params)

        return self._get_params(updated_base_class_params)

    def transform_model_params(self, unconstrained):

        unconstr_base_class_params = self._get_base_class_params(unconstrained)

        constr_base_class_params = super(KimYoo1995Model,
                self).transform_model_params(unconstr_base_class_params)

        return self._get_params(constr_base_class_params)

    def untransform_model_params(self, constrained):

        constr_base_class_params = self._get_base_class_params(constrained)

        unconstr_base_class_params = super(KimYoo1995Model,
                self).untransform_model_params(constr_base_class_params)

        return self._get_params(unconstr_base_class_params)

    def update(self, params, **kwargs):

        dtype = self.ssm.dtype
        k_regimes = self.k_regimes
        k_states = self.k_states

        base_class_params = self._get_base_class_params(params)

        super(KimYoo1995Model, self).update(base_class_params, **kwargs)

        # Filter initialization.

        initial_state = np.zeros((k_regimes, k_states), dtype=dtype)

        state_intercept = self['state_intercept']

        transition = self['transition'][0]

        raveled_state_cov = self['state_cov'][0].reshape(-1, 1)

        for i in range(k_regimes):
            initial_state[i] = np.linalg.inv(np.identity(k_states,
                    dtype=dtype) - transition).dot(state_intercept[i]
                    ).ravel()

        transition_outer_sqr = np.zeros((k_states * k_states,
                k_states * k_states), dtype=dtype)

        for i in range(k_states):
            for j in range(k_states):
                transition_outer_sqr[i * k_states:i * k_states + k_states,
                        j * k_states:j * k_states + k_states] = \
                        transition * transition[i, j]

        initial_state_cov = np.linalg.inv(np.identity(k_states * k_states,
                dtype=dtype) - transition_outer_sqr).dot(raveled_state_cov
                ).reshape(k_states, k_states).T

        self.initialize_known(initial_state, initial_state_cov)



# Trainings-Daten sourcen

## Transformieren

# Model fitten

In [ ]:
# Chauvet / Piget
k_regimes = 2
k_factors = 1
factor_order = ...
obs = ... # data